## 1. eng-fra.txt download

In [4]:
!wget https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt

--2023-03-10 02:31:13--  https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9541158 (9.1M) [text/plain]
Saving to: 'eng-fra.txt.1'

eng-fra.txt.1       100%[===================>]   9.10M  6.48MB/s    in 1.4s    

2023-03-10 02:31:15 (6.48 MB/s) - 'eng-fra.txt.1' saved [9541158/9541158]



## 2. Preprocessing
1. 필요한 없는 부호 또는 물음표, 느낌표 모으기
2. 필요한 없는 부호 ' '로 바꿔주기
3. 영어문장 불어문장으로 나누기

In [5]:
eng_sen = "Have you had dinner?"
fra_sen = "Avez-vous déjà diné?"

eng_sen = eng_sen.replace('-',' ').replace('?',' ')
fra_sen = fra_sen.replace('-',' ').replace('?',' ')

print(eng_sen.lower().split())
print(fra_sen.lower().split())

['have', 'you', 'had', 'dinner']
['avez', 'vous', 'déjà', 'diné']


In [6]:
with open('eng-fra.txt', 'r', encoding='utf-8') as f:
    text = f.read()
specials = set(text.lower()) - set('abcdefghijklmnopqrstuvwxyzàâçèéêëîïòôöùúûœас\t\n\' ')
print(sorted(specials))
print(''.join(specials))

['!', '"', '$', '%', '&', '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '~', '\xa0', '«', '\xad', '»', '\u2009', '\u200b', '‐', '–', '‘', '’', '…', '\u202f', '‽', '₂', '€', '\u3000']
‘)~5/ ,0!%"»-­　&‐6?–‽9₂ 13(+​… .7:824€;«’$


In [7]:
print(text[:100])

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Sto


In [8]:
text = text.lower()
text_alpha = text
for s in specials:
    text_alpha = text_alpha.replace(s, ' ')

print(text_alpha[:100])

go 	va  
run 	cours  
run 	courez  
wow 	ça alors  
fire 	au feu  
help 	à l'aide  
jump 	saute 
sto


In [9]:
lines = [line.split('\t') for line in text_alpha.split('\n')]
print(lines[:10])
print(len(lines))

[['go ', 'va  '], ['run ', 'cours  '], ['run ', 'courez  '], ['wow ', 'ça alors  '], ['fire ', 'au feu  '], ['help ', "à l'aide  "], ['jump ', 'saute '], ['stop ', 'ça suffit  '], ['stop ', 'stop  '], ['stop ', 'arrête toi  ']]
135843


In [10]:
import random
lines = lines[:50000]
random.shuffle(lines)
eng_sentence, fra_sentence = zip(*lines)
eng_sentence = [s.strip() for s in eng_sentence]
fra_sentence = [s.strip() for s in fra_sentence]

## 3-1. Implementation for seq2seq (Character level)

### 1. Tokenization

In [11]:
fra_sentence = [f'\t{s}\n' for s in fra_sentence]

In [12]:
eng_char = sorted(set(''.join(eng_sentence)))
fra_char = sorted(set(''.join(fra_sentence)))
eng_length = max(map(len,eng_sentence))
fra_length = max(map(len,fra_sentence))
print(eng_length,fra_length)
print(eng_char,len(eng_char))
print(fra_char,len(fra_char))

24 73
[' ', "'", 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é'] 29
['\t', '\n', ' ', "'", 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'œ', 'с'] 44


### 2. set length same using padding

In [13]:
eng_tok = {c: i for i,c in enumerate(eng_char)}
fra_tok = {c: i for i,c in enumerate(fra_char)}

print(eng_tok)
print(fra_tok)

{' ': 0, "'": 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, 'é': 28}
{'\t': 0, '\n': 1, ' ': 2, "'": 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29, 'à': 30, 'â': 31, 'ç': 32, 'è': 33, 'é': 34, 'ê': 35, 'ë': 36, 'î': 37, 'ï': 38, 'ô': 39, 'ù': 40, 'û': 41, 'œ': 42, 'с': 43}


In [14]:
eng_pad = [list(eng.ljust(eng_length)) for eng in eng_sentence]
fra_pad = [list(fra.ljust(fra_length)) for fra in fra_sentence]

### 3. change token into idx

In [15]:
import numpy as np
eng_idx = np.array([list(map(eng_tok.get, eng)) for eng in eng_pad])
fra_idx = np.array([list(map(fra_tok.get, fra)) for fra in fra_pad])

In [16]:
print(eng_idx.shape)
print(fra_idx.shape)

(50000, 24)
(50000, 73)


### 4. change the idx value into one-hot encoding

In [17]:
eng_onehot = np.array([np.eye(len(eng_char))[eng] for eng in eng_idx])
fra_onehot = np.array([np.eye(len(fra_char))[fra] for fra in fra_idx])
print(eng_onehot.shape)
print(fra_onehot.shape)

(50000, 24, 29)
(50000, 73, 44)


### implement Model

import Pytorch

In [18]:
import torch
import torch.nn as nn

Implement Encoder with class

In [37]:
encoder = nn.GRU(29, 256, batch_first= True)
output, hidden = encoder(torch.FloatTensor(eng_onehot[:10,:,:]))

print(hidden.shape)

# decoder = nn.Sequential(
#     nn.GRU(44, 256, batch_first= True),
#     nn.Linear(256, 44),
#     nn.LogSoftmax(dim=1)
# )
decoder = nn.GRU(44, 256, batch_first= True)
print(fra_onehot[:10].shape)
output, hidden = decoder(torch.FloatTensor(fra_onehot[:10]))


torch.Size([1, 10, 256])
(10, 73, 44)


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.gru = nn.GRU(self.input_dim, self.hidden_dim, num_layers= self.num_layers)

    def forward(self, x):
        outputs, hidden = self.gru(x)
        return outputs, hidden

Implement Decoder with class

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, input_dim, num_layers):
        super(Decoder, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.gru = nn.GRU(self.input_dim, self.hidden_dim, num_layers= self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, hidden):
        input = input(1, -1)
        output, hidden = self.gru(input, hidden)
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden

Implement seq2seq using `Encoder` class and `Decoder` class